In [ ]:
from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import json
import re
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [ ]:
!pip install transformers
clear_output()

In [ ]:
!pip install kss hanja
clear_output()

In [ ]:
import kss
import hanja

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Text preprocessing
- 

### 데이터 로드

In [ ]:
# csv에서 list of strings로 변환
# df = pd.read_csv('/content/drive/MyDrive/test/inter/interview.csv')
# lines = pd.Series(df['text'])

In [ ]:
# txt에서 list of strings로 변환
with open('/content/drive/MyDrive/jaso_data/interview6.txt') as f:
    lines = f.readlines()
lines = pd.Series(lines)

In [ ]:
# pickle로 저장된 list 불러오기
# import pickle
# with open("/content/drive/MyDrive/jaso_data/198000_200000.txt", "rb") as fp:
#     lines = pickle.load(fp)
# lines = [item for sublist in lines for item in sublist]
# lines = pd.Series(lines)

### 전처리

In [ ]:
# 줄바꿈, 리턴 지우기
lines = lines.apply(lambda x: x.replace('\r', '').replace('\n', ''))
lines = lines[lines!=''].reset_index(drop=True)

# <, > 특수문자 지우기
lines = lines.apply(lambda x: re.sub('[<>]', '', x))

# 3번 이상 반복되는 특수문자/문자/숫자는 마스킹 처리하기
lines = lines.apply(lambda x: re.sub('(([a-zA-Z0-9*○])\\2{2,})', '<unused0>', x))

# 괄호 안 내용 및 괄호 지우기
lines = lines.apply(lambda x: re.sub(r'\([^)]*\)', '', x))

# 기타 특수문자 지우기
lines = lines.apply(lambda x: re.sub('[\'\"!‘’@#$`%^&*\{\}\|\;?]', '', x))

# 한자 => 한글로 바꾸기
lines = lines.apply(lambda x: hanja.translate(x, 'substitution'))
# 혹시 모르니 남은 한자 모두 지우기
lines = lines.apply(lambda x: re.sub('[一-龥]', '', x))

# 공백 2개/3개 된거 하나로
lines = lines.apply(lambda x: x.replace('  ', ' '))
lines = lines.apply(lambda x: x.replace('   ', ' '))

# 동그라미 문자 2개로 마스킹된 거 '○○'로 통일
lines = lines.apply(lambda x: x.replace('oo', '○○'))
lines = lines.apply(lambda x: x.replace('OO', '○○'))
lines = lines.apply(lambda x: x.replace('ㅇㅇ', '○○'))
lines = lines.apply(lambda x: x.replace('○○', '○○'))

##### 1안 문장으로 분리 kss
lines = kss.split_sentences(lines, max_recover_length=50000, max_recover_step=20)

# Series to list
lines = lines.to_list()

# into one list
lines = [item for sublist in lines for item in sublist]

In [ ]:
##### 2안 split with '.'
new_lines = []
for line in lines:
    new_lines += line.split('. ')

lines = pd.Series(new_lines)

# 문장 양쪽 공백 제거
lines = lines.apply(lambda x: x.strip())

lines = lines.apply(lambda x: np.nan if x == '' else x)
lines = lines.dropna()

# Series to list
lines = lines.to_list()

In [ ]:
len(lines)

20978

In [ ]:
###### 추가전처리
temp = temp.apply(lambda x: re.sub('좋은점 \d', '', x))
temp = temp.apply(lambda x: re.sub('아쉬운점 \d', '', x))

In [ ]:
lines = temp

In [ ]:
# 한자 => 한글로 바꾸기
lines = lines.apply(lambda x: hanja.translate(x, 'substitution'))
# 혹시모르니 남은 한자 모두 지우기
lines = lines.apply(lambda x: re.sub('[一-龥]', '', x))

# 공백 2개/3개 된거 하나로
lines = lines.apply(lambda x: x.replace('  ', ' '))
lines = lines.apply(lambda x: x.replace('   ', ' '))

In [ ]:
# 줄바꿈, 리턴 지우기
lines = lines.apply(lambda x: x.replace('\r', '').replace('\n', ''))
lines = lines[lines!=''].reset_index(drop=True)

In [ ]:
# 특수문자 지우기
lines = lines.apply(lambda x: re.sub('[\'\"\˝\˝\-\=\→\■\!‘’@#$`%^&*\{\}\|\;?]', '', x))

In [ ]:
# unused to masking
lines = lines.apply(lambda x: x.replace('<unused0>', '○○○'))

In [ ]:
# 몇몇 기업 이름들
lines = lines.apply(lambda x: x.replace('CJ제일제당', '○○○'))
lines = lines.apply(lambda x: x.replace('삼성전자', '○○○'))
lines = lines.apply(lambda x: x.replace('삼성물산', '○○○'))
lines = lines.apply(lambda x: x.replace('포스코', '○○○'))
lines = lines.apply(lambda x: x.replace('현대자동차', '○○○'))
lines = lines.apply(lambda x: x.replace('신한은행', '○○○'))

In [ ]:
# 공백 2개/3개 된거 하나로
lines = lines.apply(lambda x: x.replace('  ', ' '))
lines = lines.apply(lambda x: x.replace('   ', ' '))

In [ ]:
lines = lines.apply(lambda x: x.strip())

### 텍스트파일
> **여기 아래 있는 텍스트 파일들은 모두 kss split 해놓고 '더블리스트' 되어 있던 걸 하나의 리스트로 합친 파일이다**

In [ ]:
#### 일단 리스트대로 한번 저장! ####
import pickle
with open('/content/drive/MyDrive/final_list_pickle222.txt', 'wb') as fp:
    pickle.dump(lines, fp)
#### #### #### #### #### ####

In [ ]:
import pickle
with open("/content/drive/MyDrive/gpt_training/final_list_pickle222.txt", "rb") as fp:
    temptemp = pickle.load(fp)

In [ ]:
temptext = ' '.join(str(e) for e in temptemp)

In [ ]:
# 텍스트 파일로 저장
f = open('/content/drive/MyDrive/interview_final222.txt', 'w')
f.write(temptext)
f.close()

In [ ]:
len(temptext)

8859475

In [ ]:
temptemp[:100]

0     학창 시절부터 줄곧 리더 역할을 많이 했습니다 하지만 역설적이게도 저의 이미지는 조...
1                      조용한 리더 이 말은 저를 가장 잘 나타내고 있는 말입니다
2                                     강인한 리더는 부러지기 쉽습니다
3                                반대로 부드러운 리더는 휘둘리기 쉽습니다
4     이 둘의 장점을 섞어 강인한 마음을 부드럽게 표현할 수 있다면 최고의 리더가 될 수...
                            ...                        
95    그래서 왕복 4시간 거리의 학교를 기숙사 생활이 아닌 통학을 하며 다녔고 아르바이트...
96             그러다 보니 체력적으로 힘에 부쳤고 이는 정신적인 스트레스로 이어졌습니다
97                        그러던 도중 어머니가 투병 생활을 시작하게 되셨습니다
98      갑작스러운 병으로 인해 병원을 오가며 처음으로 어머니의 부재 속에 생활하게 되었습니다
99    다행히 건강을 되찾으셨지만 저는 심적으로 매우 지쳤고 이때 슬럼프에 빠지지 않도록 ...
Length: 100, dtype: object

## Train test split

In [ ]:
# train test split
train = temptemp[:-int(len(temptemp)*0.1)]
test = temptemp[-int(len(temptemp)*0.1):]

# # transform to one list
# train = [item for sublist in train for item in sublist]
# test = [item for sublist in test for item in sublist]

# to string
train_ = ' '.join(str(e) for e in train)
test_ = ' '.join(str(e) for e in test)

In [ ]:
# 텍스트 파일로 저장
f = open('/content/drive/MyDrive/interview_final_train222.txt', 'w')
f.write(train_)
f.close()

f = open('/content/drive/MyDrive/interview_final_test222.txt', 'w')
f.write(test_)
f.close()

# ** 여기부터 학습루프 **


## 경로 설정

In [ ]:
# 경로 설정
train_path = '/content/drive/MyDrive/interview_final_train.txt'
test_path = '/content/drive/MyDrive/interview_final_test.txt'

## Load model / tokenizer

In [ ]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
# model.to('cuda')

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    "skt/kogpt2-base-v2",
    bos_token='<s>', eos_token='</s>', unk_token='<unk>',
    pad_token='<pad>', mask_token='<mask>', additional_special_tokens=['<unused0>']) # <unused0> for proper nouns

Downloading:   0%|          | 0.00/2.69M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


## Dataset

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=256)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=256)
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)
    
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
[Korean Sentence Splitter]: Attempting to acquire lock 140018251111632 on /content/drive/MyDrive/cached_lm_PreTrainedTokenizerFast_256_interview_final_train.txt.lock
[Korean Sentence Splitter]: Lock 140018251111632 acquired on /content/drive/MyDrive/cached_lm_PreTrainedTokenizerFast_256_interview_final_train.txt.lock
Loading features from cached file /content/drive/MyDrive/cached_lm_PreTrainedTokenizerFast_256_interview_final_train.txt [took 0.788 s]
[Korean Sentence Splitter]: Attempting to release lock 140018251111632 on /content/drive/MyDrive/cached_lm_PreTrainedTokenizerFast_256_interview_final

## Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/kogpt_checkpoints", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=15, # number of training epochs
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=24,  # batch size for evaluation
    eval_steps=400, # Number of update steps between two evaluations.
    save_steps=2000, # after # steps model is saved 
    warmup_steps=1000,# number of warmup steps for learning rate scheduler
    learning_rate=3e-5,
    prediction_loss_only=True,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()
trainer.save_model()

***** Running training *****
  Num examples = 12521
  Num Epochs = 15
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 23490


Step,Training Loss
500,3.943500
1000,3.765600
1500,3.675200
2000,3.461500
2500,3.429400
3000,3.394300
3500,3.210100
4000,3.157600
4500,3.136300
5000,3.010100


Saving model checkpoint to /content/drive/MyDrive/kogpt_checkpoints/checkpoint-2000
Configuration saved in /content/drive/MyDrive/kogpt_checkpoints/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/kogpt_checkpoints/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/kogpt_checkpoints/checkpoint-4000
Configuration saved in /content/drive/MyDrive/kogpt_checkpoints/checkpoint-4000/config.json
Model weights saved in /content/drive/MyDrive/kogpt_checkpoints/checkpoint-4000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/kogpt_checkpoints/checkpoint-6000
Configuration saved in /content/drive/MyDrive/kogpt_checkpoints/checkpoint-6000/config.json
Model weights saved in /content/drive/MyDrive/kogpt_checkpoints/checkpoint-6000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/kogpt_checkpoints/checkpoint-8000
Configuration saved in /content/drive/MyDrive/kogpt_checkpoints/checkpoint-8000/config.json
Mo

TrainOutput(global_step=23490, training_loss=2.5569751179233515, metrics={'train_runtime': 7641.7583, 'train_samples_per_second': 24.577, 'train_steps_per_second': 3.074, 'total_flos': 2.453728149504e+16, 'train_loss': 2.5569751179233515, 'epoch': 15.0})

## Save and Load

In [ ]:
# 아래 두 가지 코드로 저장된 파일들을 한 디렉토리에 합치면 된다.
# model.save_pretrained(경로)
# tokenizer.save_pretrained(경로)

# 불러올 때는
# model = GPT2LMHeadModel.from_pretrained(경로)
# tokenizer = PreTrainedTokenizerFast.from_pretrained(경로)